# 基准测试

我们测试：
* pymatching
* beliefmatching
* stimbposd
* MLD

## 导入STIM含噪线路数据

In [94]:
import stim

def read_circuit(d : int, r : int)->stim.Circuit:
    """根据相对路径, 读取stim格式的噪声电路.

    Args:
        d (int): 码矩
        r (int): 轮次

    Returns:
        _type_: _description_
    """
    if d==3:
        if r<10:
            circuit_noisy = stim.Circuit.from_file(f"./data/surface_code_bZ_d3_r0{r}_center_3_5/circuit_noisy.stim")
        else:
            circuit_noisy = stim.Circuit.from_file(f"./data/surface_code_bZ_d3_r{r}_center_3_5/circuit_noisy.stim")
    elif d==5:
        if r<10:
            circuit_noisy = stim.Circuit.from_file(f"./data/surface_code_bZ_d5_r0{r}_center_5_5/circuit_noisy.stim")
        else:
            circuit_noisy = stim.Circuit.from_file(f"./data/surface_code_bZ_d5_r{r}_center_5_5/circuit_noisy.stim")    
    return circuit_noisy

读取噪声模型

In [95]:
# noisy_circuits = [read_circuit(d,r) for d in distances for r in rounds]
noisy_circuits = [read_circuit(3, 1), read_circuit(3, 3), read_circuit(5, 1), read_circuit(5, 3)]
detector_error_models = [c.detector_error_model(flatten_loops=True) for c in noisy_circuits]
decomposed_detector_error_models = [c.detector_error_model(decompose_errors=True, flatten_loops=True) for c in noisy_circuits]

上面仅仅说明如何读取文件。

## 基准测试

In [115]:
import pymatching
from beliefmatching import BeliefMatching
from stimbposd import BPOSD

from MLD import MaxLikelihoodDecoder

import numpy as np

from typing import List, Union, Tuple

import time

def tuple_syndromes_2_str_syndromes(tuples_list):
    string_list = [''.join(map(str, tpl)) for tpl in tuples_list]
    return string_list

def array_syndromes_2_str_syndromes(tuples_array):
    # 将数组转换为字符串数组
    return np.array([''.join(map(str, tpl)) for tpl in tuples_array])
    # return np.char.join('', tuples_array.astype(str))

def experiment_benchmark_decoder(d: int, r: int, num_shots: int, seed = int)-> Tuple[List[int], np.array]:
    # 控制问题的规模
    ## 读取circuit
    # print(f"distance: {d}, round: {r}")
    noisy_circuit = read_circuit(d,r)
    detector_error_model = noisy_circuit.detector_error_model(flatten_loops=True)
    detector_number = detector_error_model.num_detectors
    decomposed_detector_error_model = noisy_circuit.detector_error_model(decompose_errors = True)
    # print(f"detector number: {detector_number}")
    # print(f"detector error model: {detector_error_model}")
    # print(f"decomposed detector error model: {decomposed_detector_error_model}")
    
    # 配置解码器
    ## MLD
    mld = MaxLikelihoodDecoder(detector_error_model = detector_error_model)
    ## MWPM
    mwpm = pymatching.Matching.from_detector_error_model(decomposed_detector_error_model)
    ## BP-Matching
    bm = BeliefMatching(noisy_circuit, max_bp_iters=20)
    ## BP-OSD    
    bposd = BPOSD(detector_error_model, max_bp_iters=20)
    
    # 随机采样
    sampler = noisy_circuit.compile_detector_sampler(seed = seed)
    syndromes_array = sampler.sample(shots=num_shots, append_observables=True)
    # 数据分离
    syndrome_number = noisy_circuits[0].detector_error_model().num_detectors
    shots, observables =  syndromes_array[:, :detector_number], syndromes_array[:, detector_number:]
    # mld输入数据
    # mld_syndrome_str = array_syndromes_2_str_syndromes(np.array(syndromes_array, dtype=int))
    
    # print("------sampler finish------")
    
    # 解码算法
    ## MLD
    # mld_start_time = time.time()
    # mld_predicted_observables = mld.decode(mld_syndrome_str)
    # mld_end_time = time.time()
    
    # print("------mld finish------")
    ## PyMatching (MWPM)
    mwpm_start_time = time.time()
    mwpm_predicted_observables = mwpm.decode_batch(shots)
    mwpm_end_time = time.time()
    
    # print("------pymatching finish------")
    ## BP-Matching
    bm_start_time = time.time()
    bm_predicted_observables = bm.decode_batch(shots)
    bm_end_time = time.time()
    
    ## BP-OSD 解码
    bposd_start_time = time.time()
    bposd_predicted_observables = bposd.decode_batch(shots)
    bposd_end_time = time.time()
    # print("------BP-OSD finish------")
    
    # 基于mld解码得到的逻辑错误率应该为：
    # mld_num_mistakes = np.sum(np.any(mld_predicted_observables != observables, axis=1))
    mwpm_num_mistakes = np.sum(np.any(mwpm_predicted_observables != observables, axis=1))
    bm_num_mistakes = np.sum(np.any(bm_predicted_observables != observables, axis=1))
    bposd_num_mistakes = np.sum(np.any(bposd_predicted_observables != observables, axis=1))
    
    # mld_logical_probability = mld_num_mistakes/num_shots
    mwpm_logical_probability = mwpm_num_mistakes/num_shots
    bm_logical_probability = bm_num_mistakes/num_shots
    bposd_logical_probability = bposd_num_mistakes/num_shots
    
    # print("------compute logical error finish------")
    
    # print(f"MLD: {mld_logical_probability}")
    print(f"PyMatching: {mwpm_logical_probability}")
    print(f"BP-Matching: {bm_logical_probability}")
    print(f"BP-OSD: {bposd_logical_probability}")
    
    # print(f"MLD time: {mld_end_time - mld_start_time}s per {num_shots} shots")
    print(f"MWPM time: {mwpm_end_time - mwpm_start_time}s per {num_shots} shots")
    print(f"BM time: {bm_end_time - bm_start_time}s per {num_shots} shots")
    print(f"BP OSD time: {bposd_end_time - bposd_start_time}s per {num_shots} shots")
    
    print("------benchmark finish------")
    
    return mwpm_logical_probability, bm_logical_probability, bposd_logical_probability

mwpm_result = 0
bm_result = 0
bposd_result = 0
for i in range(10):
    mwpm_logical_probability, bm_logical_probability, bposd_logical_probability = experiment_benchmark_decoder(d=3, r=3, num_shots= 10000, seed=i)
    mwpm_result += mwpm_logical_probability
    bm_result += bm_logical_probability
    bposd_result += bposd_logical_probability
print(mwpm_result/10, bm_result/10, bposd_result/10)

PyMatching: 0.084
BP-Matching: 0.0744
BP-OSD: 0.0731
MWPM time: 0.010591983795166016s per 10000 shots
BM time: 0.9978463649749756s per 10000 shots
BP OSD time: 5.1471946239471436s per 10000 shots
------benchmark finish------
PyMatching: 0.0846
BP-Matching: 0.0772
BP-OSD: 0.0759
MWPM time: 0.010437250137329102s per 10000 shots
BM time: 1.0645911693572998s per 10000 shots
BP OSD time: 5.097985506057739s per 10000 shots
------benchmark finish------
PyMatching: 0.0848
BP-Matching: 0.0764
BP-OSD: 0.074
MWPM time: 0.01000213623046875s per 10000 shots
BM time: 1.0115976333618164s per 10000 shots
BP OSD time: 5.233927965164185s per 10000 shots
------benchmark finish------
PyMatching: 0.0849
BP-Matching: 0.0789
BP-OSD: 0.0779
MWPM time: 0.010003805160522461s per 10000 shots
BM time: 1.0527315139770508s per 10000 shots
BP OSD time: 5.27271842956543s per 10000 shots
------benchmark finish------
PyMatching: 0.0841
BP-Matching: 0.0768
BP-OSD: 0.0758
MWPM time: 0.012002706527709961s per 10000 shots


MLD time: 40.80648636817932s per 30000 shots
MWPM time: 0.006999969482421875s per 30000 shots
BM time: 0.5118021965026855s per 30000 shots
BP OSD time: 0.45923829078674316s per 30000 shots

MLD time: 14.013275861740112s per 10000 shots
MWPM time: 0.0030450820922851562s per 10000 shots
BM time: 0.16950583457946777s per 10000 shots
BP OSD time: 0.17462468147277832s per 10000 shots

MLD的方法，利用另一种方式来计算。

In [111]:
# def experiment_ml_decoder(d: int, r: int, max_syndrome_number: Union[int, None] = None)-> Tuple[List[int], np.array]:
#     # 控制问题的规模
#     ## 读取circuit
#     print(f"distance: {d}, round: {r}")
#     noisy_circuit = read_circuit(d,r)
#     detector_error_model = noisy_circuit.detector_error_model()
#     detector_number = detector_error_model.num_detectors
#     decomposed_detector_error_model = noisy_circuit.detector_error_model(decompose_errors = True)
#     print(f"detector number: {detector_number}")
    
#     # ml和matching解码器
#     ml_decoder = MaxLikelihoodDecoder(detector_error_model = detector_error_model)
        
#     possible_syndromes_array: np.ndarray
#     possible_syndromes_str: List[str]
    
#     ## 生成可能的syndrome输入
#     if max_syndrome_number is None:
#         possible_syndromes_array = np.indices((2,) * detector_number).reshape(detector_number, -1).T
#         possible_syndromes_str = array_syndromes_2_str_syndromes(possible_syndromes_array)
#     else:
#         possible_syndromes_array = np.indices((2,) * detector_number).reshape(detector_number, -1).T[:max_syndrome_number]
#         possible_syndromes_str = array_syndromes_2_str_syndromes(possible_syndromes_array)
#     # print(possible_syndromes_array, possible_syndromes_str)
#     ## 进行MLD解码
#     ml_correcation = ml_decoder.decode(possible_syndromes_str)
    
#     #基于mld解码得到的逻辑错误率应该为：
#     mld_logical_probability = ml_decoder.compute_correcation_logical_probability(possible_syndromes_str)
#     print(f"mld_logical_probability: {mld_logical_probability}")
#     print(f"MLD decoding: {ml_correcation}, type: {type(ml_correcation)}")
#     return mld_logical_probability

# mld_logical_probability = experiment_ml_decoder(d=3, r=1)

In [98]:
# mld_logical_probability = experiment_ml_decoder(d=3, r=3)

In [99]:
0.16969999999999996/10


0.016969999999999996

In [100]:
0.17109999999999992/10


0.017109999999999993

In [101]:
0.17098999999999995/10


0.017098999999999996

In [109]:
# import pymatching
# from beliefmatching import BeliefMatching
# from stimbposd import BPOSD

# from MLD import MaxLikelihoodDecoder

# import numpy as np

# from typing import List, Union, Tuple

# import time

# def tuple_syndromes_2_str_syndromes(tuples_list):
#     string_list = [''.join(map(str, tpl)) for tpl in tuples_list]
#     return string_list

# def array_syndromes_2_str_syndromes(tuples_array):
#     # 将数组转换为字符串数组
#     return np.array([''.join(map(str, tpl)) for tpl in tuples_array])
#     # return np.char.join('', tuples_array.astype(str))

# def experiment_benchmark_decoder(d: int, r: int, num_shots: int, seed = int)-> Tuple[List[int], np.array]:
#     # 控制问题的规模
#     ## 读取circuit
#     # print(f"distance: {d}, round: {r}")
#     noisy_circuit = read_circuit(d,r)
#     detector_error_model = noisy_circuit.detector_error_model(flatten_loops=True)
#     detector_number = detector_error_model.num_detectors
#     decomposed_detector_error_model = noisy_circuit.detector_error_model(decompose_errors = True)
#     # print(f"detector number: {detector_number}")
#     # print(f"detector error model: {detector_error_model}")
#     # print(f"decomposed detector error model: {decomposed_detector_error_model}")
    
#     # 配置解码器
#     ## MLD
#     mld = MaxLikelihoodDecoder(detector_error_model = detector_error_model)
#     ## MWPM
#     mwpm = pymatching.Matching.from_detector_error_model(decomposed_detector_error_model)
#     ## BP-Matching
#     bm = BeliefMatching(noisy_circuit, max_bp_iters=20)
#     ## BP-OSD    
#     bposd = BPOSD(detector_error_model, max_bp_iters=20)
    
#     # 随机采样
#     sampler = noisy_circuit.compile_detector_sampler(seed = seed)
#     syndromes_array = sampler.sample(shots=num_shots, append_observables=True)
#     # 数据分离
#     syndrome_number = noisy_circuits[0].detector_error_model().num_detectors
#     shots, observables =  syndromes_array[:, :detector_number], syndromes_array[:, detector_number:]
#     # mld输入数据
#     mld_syndrome_str = array_syndromes_2_str_syndromes(np.array(syndromes_array, dtype=int))
    
#     print("------sampler finish------")
    
#     # 解码算法
#     ## MLD
#     mld_start_time = time.time()
#     mld_predicted_observables = mld.decode(mld_syndrome_str)
#     mld_end_time = time.time()
    
#     print("------mld finish------")
#     ## PyMatching (MWPM)
#     mwpm_start_time = time.time()
#     mwpm_predicted_observables = mwpm.decode_batch(shots)
#     mwpm_end_time = time.time()
    
#     print("------pymatching finish------")
#     ## BP-Matching
#     bm_start_time = time.time()
#     bm_predicted_observables = bm.decode_batch(shots)
#     bm_end_time = time.time()
    
#     ## BP-OSD 解码
#     bposd_start_time = time.time()
#     bposd_predicted_observables = bposd.decode_batch(shots)
#     bposd_end_time = time.time()
#     print("------BP-OSD finish------")
    
#     # 基于mld解码得到的逻辑错误率应该为：
#     mld_num_mistakes = np.sum(np.any(mld_predicted_observables != observables, axis=1))
#     mwpm_num_mistakes = np.sum(np.any(mwpm_predicted_observables != observables, axis=1))
#     bm_num_mistakes = np.sum(np.any(bm_predicted_observables != observables, axis=1))
#     bposd_num_mistakes = np.sum(np.any(bposd_predicted_observables != observables, axis=1))
    
#     mld_logical_probability = mld_num_mistakes/num_shots
#     mwpm_logical_probability = mwpm_num_mistakes/num_shots
#     bm_logical_probability = bm_num_mistakes/num_shots
#     bposd_logical_probability = bposd_num_mistakes/num_shots
    
#     print("------compute logical error finish------")
    
#     print(f"MLD: {mld_logical_probability}")
#     print(f"PyMatching: {mwpm_logical_probability}")
#     print(f"BP-Matching: {bm_logical_probability}")
#     print(f"BP-OSD: {bposd_logical_probability}")
    
#     print(f"MLD time: {mld_end_time - mld_start_time}s per {num_shots} shots")
#     print(f"MWPM time: {mwpm_end_time - mwpm_start_time}s per {num_shots} shots")
#     print(f"BM time: {bm_end_time - bm_start_time}s per {num_shots} shots")
#     print(f"BP OSD time: {bposd_end_time - bposd_start_time}s per {num_shots} shots")
    
#     print("------benchmark finish------")
    
#     return mwpm_logical_probability, bm_logical_probability, bposd_logical_probability

# mwpm_result = 0
# bm_result = 0
# bposd_result = 0
# for i in range(10):
#     mwpm_logical_probability, bm_logical_probability, bposd_logical_probability = experiment_benchmark_decoder(d=5, r=1, num_shots= 30, seed=i)
#     mwpm_result += mwpm_logical_probability
#     bm_result += bm_logical_probability
#     bposd_result += bposd_logical_probability
# print(mwpm_result/10, bm_result/10, bposd_result/10)

In [110]:
# mwpm_logical_probability, bm_logical_probability, bposd_logical_probability = experiment_benchmark_decoder(d=3, r=3, num_shots= 10, seed=i)

In [116]:
import pymatching
from beliefmatching import BeliefMatching
from stimbposd import BPOSD

from MLD import MaxLikelihoodDecoder

import numpy as np

from typing import List, Union, Tuple

import time

def tuple_syndromes_2_str_syndromes(tuples_list):
    string_list = [''.join(map(str, tpl)) for tpl in tuples_list]
    return string_list

def array_syndromes_2_str_syndromes(tuples_array):
    # 将数组转换为字符串数组
    return np.array([''.join(map(str, tpl)) for tpl in tuples_array])
    # return np.char.join('', tuples_array.astype(str))

def experiment_benchmark_decoder(d: int, r: int, num_shots: int, seed = int)-> Tuple[List[int], np.array]:
    # 控制问题的规模
    ## 读取circuit
    # print(f"distance: {d}, round: {r}")
    noisy_circuit = read_circuit(d,r)
    detector_error_model = noisy_circuit.detector_error_model(flatten_loops=True)
    detector_number = detector_error_model.num_detectors
    decomposed_detector_error_model = noisy_circuit.detector_error_model(decompose_errors = True)
    # print(f"detector number: {detector_number}")
    # print(f"detector error model: {detector_error_model}")
    # print(f"decomposed detector error model: {decomposed_detector_error_model}")
    
    # 配置解码器
    ## MLD
    mld = MaxLikelihoodDecoder(detector_error_model = detector_error_model)
    ## MWPM
    mwpm = pymatching.Matching.from_detector_error_model(decomposed_detector_error_model)
    ## BP-Matching
    bm = BeliefMatching(noisy_circuit, max_bp_iters=20)
    ## BP-OSD    
    bposd = BPOSD(detector_error_model, max_bp_iters=20)
    
    # 随机采样
    sampler = noisy_circuit.compile_detector_sampler(seed = seed)
    syndromes_array = sampler.sample(shots=num_shots, append_observables=True)
    # 数据分离
    syndrome_number = noisy_circuits[0].detector_error_model().num_detectors
    shots, observables =  syndromes_array[:, :detector_number], syndromes_array[:, detector_number:]
    # mld输入数据
    # mld_syndrome_str = array_syndromes_2_str_syndromes(np.array(syndromes_array, dtype=int))
    
    # print("------sampler finish------")
    
    # 解码算法
    ## MLD
    # mld_start_time = time.time()
    # mld_predicted_observables = mld.decode(mld_syndrome_str)
    # mld_end_time = time.time()
    
    # print("------mld finish------")
    ## PyMatching (MWPM)
    mwpm_start_time = time.time()
    mwpm_predicted_observables = mwpm.decode_batch(shots)
    mwpm_end_time = time.time()
    
    # print("------pymatching finish------")
    ## BP-Matching
    bm_start_time = time.time()
    bm_predicted_observables = bm.decode_batch(shots)
    bm_end_time = time.time()
    
    ## BP-OSD 解码
    bposd_start_time = time.time()
    bposd_predicted_observables = bposd.decode_batch(shots)
    bposd_end_time = time.time()
    # print("------BP-OSD finish------")
    
    # 基于mld解码得到的逻辑错误率应该为：
    # mld_num_mistakes = np.sum(np.any(mld_predicted_observables != observables, axis=1))
    mwpm_num_mistakes = np.sum(np.any(mwpm_predicted_observables != observables, axis=1))
    bm_num_mistakes = np.sum(np.any(bm_predicted_observables != observables, axis=1))
    bposd_num_mistakes = np.sum(np.any(bposd_predicted_observables != observables, axis=1))
    
    # mld_logical_probability = mld_num_mistakes/num_shots
    mwpm_logical_probability = mwpm_num_mistakes/num_shots
    bm_logical_probability = bm_num_mistakes/num_shots
    bposd_logical_probability = bposd_num_mistakes/num_shots
    
    # print("------compute logical error finish------")
    
    # print(f"MLD: {mld_logical_probability}")
    print(f"PyMatching: {mwpm_logical_probability}")
    print(f"BP-Matching: {bm_logical_probability}")
    print(f"BP-OSD: {bposd_logical_probability}")
    
    # print(f"MLD time: {mld_end_time - mld_start_time}s per {num_shots} shots")
    print(f"MWPM time: {mwpm_end_time - mwpm_start_time}s per {num_shots} shots")
    print(f"BM time: {bm_end_time - bm_start_time}s per {num_shots} shots")
    print(f"BP OSD time: {bposd_end_time - bposd_start_time}s per {num_shots} shots")
    
    print("------benchmark finish------")
    
    return mwpm_logical_probability, bm_logical_probability, bposd_logical_probability

mwpm_result = 0
bm_result = 0
bposd_result = 0
for i in range(10):
    mwpm_logical_probability, bm_logical_probability, bposd_logical_probability = experiment_benchmark_decoder(d=5, r=1, num_shots= 10000, seed=i)
    mwpm_result += mwpm_logical_probability
    bm_result += bm_logical_probability
    bposd_result += bposd_logical_probability
print(mwpm_result/10, bm_result/10, bposd_result/10)

PyMatching: 0.0084
BP-Matching: 0.0084
BP-OSD: 0.0086
MWPM time: 0.007239818572998047s per 10000 shots
BM time: 0.316497802734375s per 10000 shots
BP OSD time: 0.5381124019622803s per 10000 shots
------benchmark finish------
PyMatching: 0.0084
BP-Matching: 0.0086
BP-OSD: 0.0081
MWPM time: 0.008001327514648438s per 10000 shots
BM time: 0.32384204864501953s per 10000 shots
BP OSD time: 0.5659589767456055s per 10000 shots
------benchmark finish------
PyMatching: 0.008
BP-Matching: 0.0079
BP-OSD: 0.0081
MWPM time: 0.0076007843017578125s per 10000 shots
BM time: 0.32798075675964355s per 10000 shots
BP OSD time: 0.579092264175415s per 10000 shots
------benchmark finish------
PyMatching: 0.0077
BP-Matching: 0.0073
BP-OSD: 0.0075
MWPM time: 0.008001327514648438s per 10000 shots
BM time: 0.3211379051208496s per 10000 shots
BP OSD time: 0.5552799701690674s per 10000 shots
------benchmark finish------
PyMatching: 0.0073
BP-Matching: 0.007
BP-OSD: 0.0072
MWPM time: 0.00899052619934082s per 10000 s